## Project Code

Preparation:

In [59]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


pd.set_option('precision', 2)                    # Setting prices to exclude decimal points

pd.set_option('display.max_columns', 500)        # Setting DataFrame columns to show all of them

pd.options.mode.chained_assignment = None        # default='warning signs', when adding new columns to data frames

from sklearn.model_selection import train_test_split     # split data into random train and test subsets

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression

Open files to dataframes:

In [47]:
# Open housing data into a dataframe
dtitles = pd.DataFrame.from_csv('Housing_data/test.csv', index_col=None)

# Open housing price data into a dataframe
dprices = pd.DataFrame.from_csv('Housing_data/sample_submission.csv', index_col=None)

# open weather data file
weather_d = pd.DataFrame.from_csv('Weather_data/weather.csv', index_col=None)           

Wrangle housing data:

In [48]:
# Merge/combine the properties' descriptions and its sales prices into a new dataframe.
h_data = pd.merge(dtitles, dprices, on='Id')

# Filtering from 2006-2009!
hs_data = h_data[(h_data.YrSold >= 2006) & (h_data.YrSold <= 2009)]                            

# Rename columns
hs_data = hs_data.rename(columns={"MoSold":"month", "YrSold":"year"})          

# Building dataframe to a series datetime index                                                                          
hs_data.loc[:, 'day'] = 1                                                       

# adding column as one datetime.
date = pd.to_datetime(hs_data[['year', 'month', 'day']])                        
hs_data.loc[:, 'Date'] = date
hs_data.loc[:, 'Date'] = hs_data.Date.dt.to_period('M')
                                                                                   
# Seting, sorting and renaming index
hs_data = hs_data.set_index('Date').sort_index()                               
hs_data.drop(['day'], axis=1, inplace=True)

# changing/rename index to string object for later joining
hs_data.index = hs_data.index.map(str).rename('Year_Month')          

Wrangle weather data and join dataframes:

In [49]:
# Sorting to data needed
weather_d1 = weather_d[weather_d.STATION_NAME == 'AMES 8 WSW IA US']                   
weather_d2 = weather_d1[(weather_d1.DATE >= 20060101) & (weather_d1.DATE <= 20091231)]
weather_d3 = weather_d2[['DATE', 'PRCP', 'SNOW', 'TMAX', 'TMIN']]

 # Getting date to datetime, setting and sortinging index to ('Date')
weather_d3.loc[:, 'Date'] = pd.to_datetime(weather_d3.DATE, format='%Y%m%d')           
weather_d4 = weather_d3[['Date', 'PRCP', 'SNOW', 'TMAX', 'TMIN']]                      

# Building and setting index to year, and month.
weather_d4['Date'] =weather_d4.Date.dt.to_period('M')                                   
weather_data = weather_d4.set_index('Date').sort_index()                                 

# Changed corrupted values to Not a Number(NaN)
weather_data[weather_data == -9999] = np.nan                                   

# Changed names for columns
col = ['Avg_Prcp', 'Avg_Snow', 'Avg_Tmax', 'Avg_Tmin']                          
weather_data.columns = col

# Getting weather averages
wth_data_avg = weather_data.groupby(weather_data.index).mean()                 

# Change index type to string object for later merging of dataframes
wth_data_avg.index = wth_data_avg.index.map(str).rename('Year_Month') 

# join new housing and weather dataframes
hs_wth_data = hs_data.join(wth_data_avg)

# changing index to dates
hs_wth_df = hs_wth_data.reset_index(level=['Year_Month'])

# Drop 'Id' column, not needed
data = hs_wth_df.drop('Id', 1)

Not a number values(NaN) and Outliers:

In [50]:
#data.isnull().sum().sort_values(ascending=False).head(30) <---> Function to see values
# Remove columns/features with more than 1000 NaN values
data = data.dropna(axis=1, thresh=1000)

# Fill float columns' NaN values with means
data[['LotFrontage', 'MasVnrArea','GarageArea', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2', 'BsmtFinSF1']] = data[
    ['LotFrontage', 'MasVnrArea','GarageArea', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2', 'BsmtFinSF1']].fillna(
    data[['LotFrontage', 'MasVnrArea','GarageArea', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2', 'BsmtFinSF1']].mean())

# Fill int columns' NaN values with mode, most frequent
data[['GarageYrBlt','BsmtFullBath','BsmtHalfBath', 'GarageCars']] = data[
    ['GarageYrBlt','BsmtFullBath','BsmtHalfBath', 'GarageCars']].fillna(
    data[['GarageYrBlt','BsmtFullBath','BsmtHalfBath', 'GarageCars']].mode().iloc[0])

#data_object_sum = data.select_dtypes(include=['object']).isnull().sum() <---> Function to see non-numerical NaN values
# Object/Categorical data NaN values droped. Keep all NaN values for float and int.
data = data.dropna(axis=0, subset=['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual',
                                  'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Functional', 'GarageType',
                                  'GarageFinish', 'GarageQual', 'GarageCond', 'SaleType'])
data.head()

,Year_Month,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,month,year,SaleType,SaleCondition,SalePrice,Avg_Prcp,Avg_Snow,Avg_Tmax,Avg_Tmin
0,2006-01,20,RL,68.86,13052,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,6,1965,1965,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,712.0,Unf,0.0,312.0,1024.0,GasA,TA,Y,SBrkr,1024,0,0,1024,0.0,0.0,1,1,3,1,TA,5,Typ,0,Attchd,1965.0,Unf,1.0,308.0,TA,TA,Y,0,0,0,0,0,0,0,1,2006,WD,Normal,183433.47,0.02,0.06,41.42,25.84
1,2006-01,60,RL,50.00,13128,Pave,IR1,HLS,AllPub,CulDSac,Gtl,Gilbert,Norm,Norm,1Fam,2Story,8,5,2005,2005,Gable,CompShg,VinylSd,VinylSd,BrkFace,216.0,Gd,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,1074.0,1074.0,GasA,Ex,Y,SBrkr,1074,990,0,2064,0.0,0.0,2,1,4,1,Gd,7,Typ,1,Attchd,2005.0,Fin,2.0,527.0,TA,TA,Y,0,119,0,0,0,0,0,1,2006,WD,Normal,196858.92,0.02,0.06,41.42,25.84
2,2006-01,80,RL,100.00,14330,Pave,IR1,Low,AllPub,Corner,Gtl,Veenker,Norm,Norm,1Fam,SLvl,7,4,1974,1974,Gable,CompShg,WdShing,Wd Sdng,BrkFace,145.0,Gd,Fa,CBlock,Gd,TA,Gd,ALQ,1023.0,BLQ,497.0,228.0,1748.0,GasA,Gd,Y,SBrkr,2151,495,0,2646,1.0,2.0,2,0,3,1,Gd,9,Mod,4,Attchd,1974.0,RFn,2.0,550.0,TA,TA,Y,641,100,0,0,0,800,0,1,2006,WD,Normal,185948.63,0.02,0.06,41.42,25.84
4,2006-01,60,RL,92.00,15922,Pave,IR1,HLS,AllPub,Corner,Gtl,StoneBr,Norm,Norm,1Fam,2Story,9,5,2005,2006,Hip,CompShg,VinylSd,VinylSd,BrkFace,550.0,Gd,TA,PConc,Ex,TA,Av,Unf,0.0,Unf,0.0,1390.0,1390.0,GasA,Ex,Y,SBrkr,1390,1405,0,2795,0.0,0.0,3,1,4,1,Ex,10,Typ,1,BuiltIn,2005.0,Fin,3.0,660.0,TA,TA,Y,272,102,0,0,0,0,0,1,2006,New,Partial,202357.62,0.02,0.06,41.42,25.84
5,2006-01,30,RL,50.00,5220,Pave,IR1,Lvl,AllPub,Inside,Gtl,SWISU,Norm,Norm,1Fam,1Story,5,3,1936,1950,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,830.0,830.0,GasA,Gd,Y,SBrkr,879,0,0,879,0.0,0.0,1,0,2,1,TA,4,Typ,0,Detchd,1936.0,Unf,1.0,180.0,TA,TA,P,0,108,0,0,0,0,0,1,2006,WD,Normal,154743.92,0.02,0.06,41.42,25.84


Data split:

In [51]:
X, y = data.drop(['Year_Month', 'SalePrice'], axis=1), data.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 5)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(819, 78)
(819,)
(351, 78)
(351,)


In [65]:
rfc = RandomForestClassifier(n_jobs=-1, max_features= 'sqrt', n_estimators=50, oob_score=True)


param_grid = {'max_features': ['auto', 'sqrt', 'log2']}


CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)

CV_rfc.fit(X_train, y_train)

print(CV_rfc.best_params_)

ValueError: could not convert string to float: 'Normal'